In [ ]:
%matplotlib inline


# Superimposition of two protein structures

This script superimposes the atom coordinates of the Ku70/80
heterodimer onto the corresponding structure bound to DNA
(superimposition of 1JEQ onto 1JEY).
The visualisation was coducted with PyMOL.

* *Orange*: Ku dimer originally bound to DNA
* *Green*:  Free Ku dimer


In [12]:
# Code source: Patrick Kunzmann
# License: BSD 3 clause


import os
from tempfile import NamedTemporaryFile
import biotite.structure as struc
import biotite.structure.io.pdbx as pdbx
import biotite.structure.io.pdb as pdb
import biotite.database.rcsb as rcsb
import numpy as np
    

ku_dna_file = NamedTemporaryFile(suffix=".cif", mode='w')
ku_file     = NamedTemporaryFile(suffix=".cif", mode='w')
# The output file names
# Modify these values for actual file output
ku_dna_file_name = os.path.join(os.path.abspath(""), "KUDNA.cif")
ku_file_name = os.path.join(os.path.abspath(""), "KU.cif")

# Download and parse structure files
ku_dna = pdbx.get_structure(pdbx.PDBxFile.read(rcsb.fetch("1IEP", "cif")))[0]
ku     = pdb.get_structure(pdb.PDBFile.read(os.path.join(os.path.abspath(""), "fold.pdb")))[0]

# Remove DNA and water
ku_dna = ku_dna[(ku_dna.chain_id == "A") | (ku_dna.chain_id == "B")]
ku_dna = ku_dna[~struc.filter_solvent(ku_dna)]
ku = ku[~struc.filter_solvent(ku)]
# The structures have a differing amount of atoms missing
# at the the start and end of the structure
# -> Find common structure
ku_dna_common = ku_dna[struc.filter_intersection(ku_dna, ku)]
ku_common = ku[struc.filter_intersection(ku, ku_dna)]
# Superimpose
ku_superimposed, transformation = struc.superimpose(
    ku_dna_common, ku_common, (ku_common.atom_name == "CA")
)
# We do not want the cropped structures
# -> apply superimposition on original structures
ku_superimposed = struc.superimpose_apply(ku, transformation)
# Write PDBx files as input for PyMOL
cif_file = pdbx.PDBxFile()
pdbx.set_structure(cif_file, ku_dna, data_block="ku_dna")
cif_file.write(ku_dna_file_name)
cif_file = pdbx.PDBxFile()
pdbx.set_structure(cif_file, ku_superimposed, data_block="ku")
cif_file.write(ku_file_name)
# Visualization with PyMOL...

ku_dna_file.close()
ku_file.close()

    A       1  PRO N      N       -11.500    2.367    7.492
    A       1  PRO CA     C       -10.297    3.156    7.203
    A       1  PRO C      C        -9.617    2.742    5.898
    A       1  PRO CB     C        -9.391    2.871    8.406
    A       1  PRO O      O        -9.586    1.556    5.566
    A       1  PRO CG     C       -10.312    2.369    9.469
    A       1  PRO CD     C       -11.422    1.599    8.812
    A       2  ILE N      N        -9.305    3.883    5.223
    A       2  ILE CA     C        -8.617    3.605    3.967
    A       2  ILE C      C        -7.191    4.148    4.031
    A       2  ILE CB     C        -9.375    4.215    2.764
    A       2  ILE O      O        -6.977    5.297    4.414
    A       2  ILE CG1    C       -10.781    3.619    2.658
    A       2  ILE CG2    C        -8.586    3.998    1.468
    A       2  ILE CD1    C       -11.680    4.328    1.656
    A       3  ALA N      N        -6.176    3.268    3.701
    A       3  ALA CA     C        -4.78